In [1]:
# libraries
import torch
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from torchmetrics import R2Score, MeanSquaredError

r2score = R2Score()
msescore = MeanSquaredError()

torch.manual_seed(2)
np.random.seed(2)
torch.set_printoptions(precision=8)

c:\Users\tln229\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Model (system)
class System(torch.nn.Module):
    def __init__(self):
        super(System, self).__init__()
        self.input_ini    = torch.nn.Linear(1, 12)
        self.input_signal = torch.nn.Linear(10,12)
        self.input_t      = torch.nn.Linear(1, 12)

    def forward(self, x_0, u, t):
        z_ini   = torch.selu(self.input_ini(x_0))
        z_input = torch.selu(self.input_signal(u))
        z_t     = torch.selu(self.input_t(t))
        z       = z_ini * z_input * z_t
        z       = torch.sum(z, dim=1).reshape(-1,1)
        return z

In [3]:
# Model error
def eval(model, testset):
    with torch.no_grad():
        pred_Y = model(testset.x0_data, testset.u_data, testset.t_data)

    r2  = r2score(pred_Y, testset.y_data)
    mse = msescore(pred_Y, testset.y_data)
    return r2.item(), mse.item()

In [4]:
# Data
class Data(torch.utils.data.Dataset):
  def __init__(self, src_file, start, stop):
    df = pd.read_csv(src_file)
    X0 = np.array(df['X0'][start:stop]).reshape(-1,1)
    U1 = np.array(df['U01'][start:stop])
    U2 = np.array(df['U02'][start:stop])
    U3 = np.array(df['U03'][start:stop])
    U4 = np.array(df['U04'][start:stop])
    U5 = np.array(df['U05'][start:stop])
    U6 = np.array(df['U06'][start:stop])
    U7 = np.array(df['U07'][start:stop])
    U8 = np.array(df['U08'][start:stop])
    U9 = np.array(df['U09'][start:stop])
    U10= np.array(df['U10'][start:stop])
    U  = np.column_stack((U1, U2, U3, U4, U5, U6, U7, U8, U9, U10))
    T  = np.array(df['T'][start:stop]).reshape(-1,1)
    Y  = np.array(df['Y'][start:stop]).reshape(-1,1)

    self.x0_data = torch.tensor(X0, dtype=torch.float32)
    self.u_data  = torch.tensor(U,  dtype=torch.float32)
    self.t_data  = torch.tensor(T,  dtype=torch.float32)
    self.y_data  = torch.tensor(Y,  dtype=torch.float32)

  def __len__(self):
    return len(self.x0_data)

  def __getitem__(self, idx):
    if torch.is_tensor(idx):
      idx = idx.tolist()
    x0  = self.x0_data[idx]
    u   = self.u_data[idx]
    t   = self.t_data[idx]
    y   = self.y_data[idx]
    sample = {'x0':x0, 'u':u, 't':t, 'y':y}
    return sample

In [5]:
# Early stopping
def early_stop(list, min_epochs, patience):
    if(len(list) > min_epochs):
        if(np.max(list[-patience:]) < 1.0001*np.max(list[0: -patience])):
            return 1
    return 0

In [6]:
# Train function
def train(net, train_ds, test_ds, lr=0.001, min_epochs=200, max_epochs=100000, patience=100):
    loss_func  = torch.nn.MSELoss()
    optimizer  = torch.optim.Adam(net.parameters(), lr=lr)

    train_ldr = torch.utils.data.DataLoader(train_ds, batch_size=train_ds.y_data.shape[0], shuffle=True)

    R2  = np.array([])
    MSE = np.array([])
    for epoch in range(0, max_epochs+1):
        net.train()
        loss  = 0
        count = 0
        for (_, batch) in enumerate(train_ldr):
            X0 = batch['x0']
            U  = batch['u']
            T  = batch['t']
            Y  = batch['y']

            optimizer.zero_grad()
            output = net(X0, U, T)             # compute the output of the Network
            loss_val = loss_func(output, Y)    # loss function
            loss += loss_val.item()            # accumulate
            loss_val.backward()                # gradients
            optimizer.step()                   # update paramters
            count += 1
        
        net.eval()
        R2  = np.append(R2, eval(net, test_ds)[0])
        MSE = np.append(MSE, eval(net, test_ds)[1])

        if(epoch%500==0):
            print("epoch = %5d \t loss = %12.4f \t R2 = %12.4f \t MSE = %12.4f" % (epoch, loss/count, eval(net, test_ds)[0], eval(net, test_ds)[1]))
        
        if(early_stop(list = R2, min_epochs = min_epochs, patience = patience) == 1):
            break
    
    return R2, MSE

In [7]:
# Create Dataset and DataLoader objects
src_file = 'training_data.csv'
train_ds = Data(src_file, 0, 15000)
test_ds  = Data(src_file, 15000, 20000)

# Create network
device = torch.device("cpu")
net = System().to(device)

# train model
lr         = 0.001
min_epochs = 500
max_epochs = 100000
patience   = 300
R2, MSE = train(net, train_ds, test_ds, lr, min_epochs, max_epochs, patience)

epoch =     0 	 loss = 9527698.0000 	 R2 = -117252.0078 	 MSE = 5185350.0000
epoch =   500 	 loss =    9429.8926 	 R2 =    -115.6842 	 MSE =    5160.1968
epoch =  1000 	 loss =    2156.9792 	 R2 =     -26.9958 	 MSE =    1238.0768
epoch =  1500 	 loss =     721.8849 	 R2 =      -8.1729 	 MSE =     405.6609
epoch =  2000 	 loss =     537.0233 	 R2 =      -6.6985 	 MSE =     340.4543
epoch =  2500 	 loss =     453.5688 	 R2 =      -5.6616 	 MSE =     294.5984
epoch =  3000 	 loss =     406.9037 	 R2 =      -5.0481 	 MSE =     267.4684
epoch =  3500 	 loss =     367.3782 	 R2 =      -4.4472 	 MSE =     240.8931
epoch =  4000 	 loss =     334.7917 	 R2 =      -3.8861 	 MSE =     216.0827
epoch =  4500 	 loss =     303.7854 	 R2 =      -3.3740 	 MSE =     193.4328
epoch =  5000 	 loss =     273.9212 	 R2 =      -2.8941 	 MSE =     172.2119
epoch =  5500 	 loss =     245.2221 	 R2 =      -2.4395 	 MSE =     152.1066
epoch =  6000 	 loss =     216.1785 	 R2 =      -2.0157 	 MSE =     133.3656

In [8]:
torch.save(net, 'system.pt')